_As one of the most diverse places in the planet, Toronto has been permeated by foods from different regions and countries. Many people love to spend their weekend or a shot vacation in Toronto, and eating would be essential and important to them, which could hugely influence their happiness. But when facing too many choices, people can possibly be in two minds and waste time in selecting a restaurant._   

__To relief the pressure brought by hundreds of thousands of information of restaurants,  I want to briefly recommend several restaurants to people, knowing their traffic mode and taste preference.__  

To reach this goal, first of all, people will be pleased to know the most common restaurant categories in their neighborhoods, because the likelihood of discovering nice restaurant might be higher.  

The second consideration must be transportation. For drivers, they would get annoyed if there are no nearby parking lot when they arrive at the restaurant; for pedestrians,  they would be exhausted if the destination is far away from bus stops or metro stations.


In [2]:
import pandas as pd
import numpy as np
import string
import requests
import geocoder
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
from sklearn.cluster import KMeans
import csv

### This project is based on Toronto, so I continue to use the neighbourhoods data I got last assignment

In [3]:
raw_data = pd.read_csv('pccan.csv')
raw_data = raw_data[~(raw_data['Borough']=='Not assigned')].reset_index(drop= True)
pd_data = raw_data.groupby(['Postcode','Borough']).sum().reset_index().set_index('Postcode')
pd_data['Neighbourhood'] = raw_data.groupby('Postcode')['Neighbourhood'].agg(lambda x:','.join(x))
embed_data = pd_data.reset_index()
embed_data.rename(columns={'Postcode':'PostalCode'},inplace = True)

In [4]:
embed_data.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
print('There are {} postalcode in the dataframe'.format(embed_data.shape[0]))

There are 103 postalcode in the dataframe


### get the latitude and longitude coordinates of each neighborhood 

In [6]:
coor = pd.read_csv('Geospatial_Coordinates.csv').set_index('Postal Code')
coor.rename(index={'Postal Code':'PostalCode'},inplace = True)

In [7]:
embed_data = embed_data.set_index('PostalCode')
toronto_data = pd.merge(embed_data,coor,left_index=True,right_index=True).reset_index()

### show the map of Toronto 

In [8]:
#geographical coordinates of Toronto
latitude = 43.653226
longitude = -79.383184

In [9]:
# create map of Torontousing latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

project outline:  
I will also search for Parking and public-transportation stations(bus stop and metro station) in each neighborhood.  
* Parking (Foursquare Category Id: '4c38df4de52ce0d596b336e1')
* Public transportation:
  * Bus station (Foursquare Category Id: '4bf58dd8d48988d1fe931735')  
  * Bus stop (Foursquare Category Id: '52f2ab2ebcbc57f1066b8b4f')  
  * Metro station (Foursquare Category Id: '4bf58dd8d48988d1fd931735')  

In [10]:
CLIENT_ID = 'AAOIDTJMJGHIQSS4CK51C2HZD52KB5U45JLQX0YIYW0BDPF3' # your Foursquare ID
CLIENT_SECRET = 'X5TCGWCVL3FSRJSUMYMOU3PJN0JZAZAAAVD4LKAV33S1VQWC' # your Foursquare Secret
VERSION = '20180605'

In [11]:
ID = ['4c38df4de52ce0d596b336e1','4bf58dd8d48988d1fe931735','52f2ab2ebcbc57f1066b8b4f','4bf58dd8d48988d1fd931735']

In [79]:
def getNearbyTransport(names,latitudes,longitudes,ID,Radius = 500,LIMIT = 100):
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url_park = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(
                CLIENT_ID,
                CLIENT_SECRET,
                lat,
                lng,
                VERSION,
                ID[0],
                Radius,
                LIMIT)
        # make the GET request
        try:
            results_park = requests.get(url_park).json()['response']['venues']
            count_park = np.shape(results_park)[0]
        except:
            count_park = 0
        try:
            url_bus1 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(
                CLIENT_ID,
                CLIENT_SECRET,
                lat,
                lng,
                VERSION,
                ID[1],
                Radius,
                LIMIT)
            # make the GET request
            results_bus1 = requests.get(url_bus1).json()['response']['venues']
            count_bus1 = np.shape(results_bus1)[0]
        except:
            count_bus1 = 0
        try:
            url_bus2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(
                CLIENT_ID,
                CLIENT_SECRET,
                lat,
                lng,
                VERSION,
                ID[2],
                Radius,
                LIMIT)
            # make the GET request
            results_bus2 = requests.get(url_bus2).json()['response']['venues']
            count_bus2 = np.shape(results_bus2)[0]
        except:
            count_bus2 = 0
        try:
            url_bus3 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(
                CLIENT_ID,
                CLIENT_SECRET,
                lat,
                lng,
                VERSION,
                ID[3],
                Radius,
                LIMIT)
            # make the GET request
            results_bus3 = requests.get(url_bus3).json()['response']['venues']
            count_bus3 = np.shape(results_bus3)[0]
        except:
            count_bus3 = 0
        count_pub = count_bus1+count_bus2+count_bus3
        venues_list.append([(name,
                           lat,
                           lng,
                           count_park,count_pub
                            )])
    nearby_transport = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    # item for (venue_list in venues_lists) for item in venue_list
    nearby_transport.columns = ['Neighbourhood',
                             'Neighbourhood Latitude',
                             'Neighbourhood Longitude',
                             'parking','public transport']
    return(nearby_transport)

In [80]:
toronto_Transport = getNearbyTransport(toronto_data['Neighbourhood'],
                                  toronto_data['Latitude'],
                                  toronto_data['Longitude'],ID)

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [89]:
toronto_Transport

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,parking,public transport
0,"Rouge,Malvern",43.806686,-79.194353,0,0
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,0,0
2,"Guildwood,Morningside,West Hill",43.763573,-79.188711,0,0
3,Woburn,43.770992,-79.216917,0,0
4,Cedarbrae,43.773136,-79.239476,0,0
5,Scarborough Village,43.744734,-79.239476,0,0
6,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,0,0
7,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,0,0
8,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476,0,0
9,"Birch Cliff,Cliffside West",43.692657,-79.264848,0,0


  
One-hot encoding will be used to transform the feature of Parking and public-tranportation stations. K-Means is chosen to be applied on this question for following reasons:
	1. We do not have labaled data
	2. 3 traffic type clusters we would like (all-friendly, driver-friendly, public-friendly)


For each traffic type,  I will cluster them by restaurant categories.  

As an addict to Asian food, I select 7 types of restaurants, including Chinese Restaurant, Hotpot Restaurant, Japanese Restaurant, Korean Restaurant, Malay Restaurant, Thai Restaurant and Vietnamese Restaurant. Each neighborhood will search for each type of these reataurant and count the number.
One-hot encoding will be used again. I want to get 3 clusters according to the restaurant categories.   
* Chinese Restaurant: '4bf58dd8d48988d145941735'    
* Hotpot Restaurant: '52af0bd33cf9994f4e043bdd'  
* Japanese Restaurant: '4bf58dd8d48988d111941735'  
* Korean Restaurant: '4bf58dd8d48988d113941735'  
* Malay Restaurant: '4bf58dd8d48988d156941735'  
* Thai Restaurant: '4bf58dd8d48988d149941735'    
* Vietnamese Restaurant: '4bf58dd8d48988d14a941735'  

Finally, according to your traffic mode and taste, you can get a list of 
suitable neighborhood,. Once you choose the idea neighborhood (probably nearest to you), I want to show you the price and likes the restaurant has got. To make an example, I want to randomly choose a neighborhood and get the details (price, rating, been Here) of a list of a certain restaurants(50) and use k-means to predict 3 cluster.